In [ ]:
import numpy as np
import muon as mu
from scipy.ndimage import gaussian_filter1d
import pyBigWig
import pandas as pd
import scipy.ndimage



In [ ]:
# read your muon data here
mdata_processed = mu.read("muon_data/pbmc10k_processed.h5mu")
mdata_processed

atac = mdata_processed.mod['atac']
atac

counts = atac.layers['counts'].toarray()
intervals = atac.var['interval']

In [ ]:
smooth_sigma = 2
smoothed_counts = scipy.ndimage.gaussian_filter1d(counts, sigma=smooth_sigma, axis=1)

In [ ]:
def smooth_counts(counts, sigma=3):
    return gaussian_filter1d(counts, sigma=sigma, axis=0)

smoothed_counts_t = smooth_counts(counts)

In [ ]:
smoothed_counts.shape

In [ ]:
gene_totals = np.sum(smoothed_counts, axis=0)
gene_totals.shape

In [ ]:
intervals_df = intervals.str.extract(r'(?P<chrom>[^:]+):(?P<start>\d+)-(?P<end>\d+)')
intervals_df['start'] = intervals_df['start'].astype(int)
intervals_df['end'] = intervals_df['end'].astype(int)

intervals_df.shape
intervals_df['smoothed_counts'] = gene_totals

In [ ]:
intervals_df

In [ ]:
intervals_df.to_csv('muon_data/atac_data.bedGraph', sep='\t', index=False, header=False)

In [ ]:
chrom_sizes = pd.read_csv('hg38.chrom.sizes', sep='\t', header=None, names=['chrom', 'size'])

In [ ]:
filtered_bedgraph = intervals_df[intervals_df['chrom'].isin(chrom_sizes['chrom'])]

# save filtered bedGraph 
filtered_bedgraph.to_csv('muon_data/filtered_atac_data.bedGraph', sep='\t', index=False, header=False)

In [ ]:
filtered_bedgraph